In [44]:
# Installing required libraries
!pip install pandas numpy scikit-learn tensorflow ta pyswarm plotly

In [45]:
import tensorflow as tf

# Checking if GPU is available
gpu_available = tf.config.list_physical_devices('GPU')

if gpu_available:
    print("GPU is available:")
    for gpu in gpu_available:
        print(f"  - {gpu}")
else:
    print("No GPU available. TensorFlow will use CPU.")

GPU is available:
  - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import ta
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from pyswarm import pso
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")

In [47]:
# Mounting Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
# Loading dataset
df = pd.read_csv('/content/drive/MyDrive/Weekly Rubber TSR20 Futures Historical Data_2015-2025.csv')

In [57]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,06/28/2015,152.3,154.6,158.0,145.0,1.14K,-2.75%
1,07/05/2015,144.9,149.0,149.5,137.9,2.47K,-4.86%
2,07/12/2015,146.6,141.1,147.0,140.5,1.17K,1.17%
3,07/19/2015,142.4,145.5,149.4,140.5,1.17K,-2.86%
4,07/26/2015,140.0,141.0,144.0,137.5,1.01K,-1.69%


In [86]:
# Reloading  dataset to ensuring original columns present
df = pd.read_csv('/content/drive/MyDrive/Weekly Rubber TSR20 Futures Historical Data_2015-2025.csv')

# Data Preprocessing
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Handling volume column
if 'Vol.' in df.columns:
    df['Volume'] = df['Vol.'].str.replace('K', '', regex=False).astype(float) * 1000
elif 'Volume' in df.columns and pd.api.types.is_numeric_dtype(df['Volume']):
    pass
else:
    raise KeyError("Neither 'Vol.' nor a valid numeric 'Volume' column found. Available columns: " + str(df.columns.tolist()))


# Handling percentage change column
if 'Change %' in df.columns:
    df['Change'] = df['Change %'].str.replace('%', '', regex=False).astype(float) / 100
elif 'Change' in df.columns and pd.api.types.is_numeric_dtype(df['Change']):
    pass
else:
    raise KeyError("Neither 'Change %' nor a valid numeric 'Change' column found. Available columns: " + str(df.columns.tolist()))

# Selecting the final desired columns, implicitly dropping other duplicates
df = df[['Date', 'Price', 'Open', 'High', 'Low', 'Volume', 'Change']]


# Outlier handling using IQR
for col in ['Price', 'Open', 'High', 'Low', 'Volume', 'Change']:
    # Ensuring the column exists and is numeric before processing
    if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        median = df[col].median()
        df[col] = df[col].where((df[col] >= lower_bound) & (df[col] <= upper_bound), median)
    else:
        print(f"Warning: Column '{col}' not found or is not numeric, skipping outlier handling.")

In [87]:
# Feature Engineering
df['MA_5'] = ta.trend.sma_indicator(df['Price'], window=5)
df['MA_10'] = ta.trend.sma_indicator(df['Price'], window=10)
df['MA_20'] = ta.trend.sma_indicator(df['Price'], window=20)
df['EMA_5'] = ta.trend.ema_indicator(df['Price'], window=5)
df['EMA_10'] = ta.trend.ema_indicator(df['Price'], window=10)
df['BB_upper'], df['BB_middle'], df['BB_lower'] = ta.volatility.bollinger_hband(df['Price'], window=20), \
                                                 ta.volatility.bollinger_mavg(df['Price'], window=20), \
                                                 ta.volatility.bollinger_lband(df['Price'], window=20)
df['BB_width'] = df['BB_upper'] - df['BB_lower']
df['RSI'] = ta.momentum.rsi(df['Price'], window=14)
df['Rolling_std_5'] = df['Price'].rolling(window=5).std()
df['Rolling_std_10'] = df['Price'].rolling(window=10).std()
df['High_Low_Spread'] = df['High'] - df['Low']
df['Price_Open_Diff'] = df['Price'] - df['Open']
df['Month'] = df['Date'].dt.month
df['Quarter'] = df['Date'].dt.quarter
df['Week_of_Year'] = df['Date'].dt.isocalendar().week

In [88]:
# Droping NaN rows
df = df.dropna()

In [89]:
# Preparing lagged features for SVR
for i in range(1, 5):
    df[f'Price_t-{i}'] = df['Price'].shift(i)

In [91]:
# Data splitting
train_size = int(len(df) * 0.7)
val_size = int(len(df) * 0.15)
train_df = df[:train_size]
val_df = df[train_size:train_size + val_size]
test_df = df[train_size + val_size:]

In [92]:
# Scale featuring
scaler = MinMaxScaler()
feature_cols = ['Price', 'Open', 'High', 'Low', 'Volume', 'Change', 'MA_5', 'MA_10', 'MA_20',
                'EMA_5', 'EMA_10', 'BB_upper', 'BB_lower', 'BB_width', 'RSI',
                'Rolling_std_5', 'Rolling_std_10', 'High_Low_Spread', 'Price_Open_Diff',
                'Month', 'Quarter', 'Week_of_Year']
scaler.fit(train_df[feature_cols])
train_scaled = scaler.transform(train_df[feature_cols])
val_scaled = scaler.transform(val_df[feature_cols])
test_scaled = scaler.transform(test_df[feature_cols])

In [93]:
# Preparing sequences for LSTM
def create_sequences(data, seq_length=12, forecast_horizon=4):
    X, y = [], []
    for i in range(len(data) - seq_length - forecast_horizon + 1):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length + forecast_horizon - 1, 0])  # Price column
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled)
X_val, y_val = create_sequences(val_scaled)
X_test, y_test = create_sequences(test_scaled)

In [94]:
# Baseline LSTM
baseline_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(12, X_train.shape[2])),
    LSTM(50),
    Dense(1)
])
baseline_model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mse')
if X_val.shape[0] > 0 and y_val.shape[0] > 0:
    print("Training baseline model with validation data...")
    baseline_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, verbose=1)
else:
    print("Validation data is empty. Training baseline model without validation data...")
    baseline_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

Training baseline model with validation data...
Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0700 - val_loss: 0.0045
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0193 - val_loss: 0.0028
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0202 - val_loss: 0.0059
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0207 - val_loss: 0.0058
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 0.0164 - val_loss: 0.0062
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0134 - val_loss: 0.0198
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0258 - val_loss: 0.0037
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0127 - val_loss: 0.0048
Epoch 9/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0222 - val_loss: 0.0112
Epoch 10/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0133 - val_loss: 0.0252
Epoch 11/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0230 - val_loss: 0.0044
Epoch 12/50
11/

In [95]:
# PSO for LSTM optimization
def lstm_objective(params):
    units_1, units_2, dropout, lr = params
    model = Sequential([
        LSTM(int(units_1), return_sequences=True, input_shape=(12, X_train.shape[2])),
        Dropout(dropout),
        LSTM(int(units_2)),
        Dropout(dropout),
        Dense(1)
    ])
    model.compile(optimizer=RMSprop(learning_rate=lr), loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', patience=10)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        epochs=50, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)
    return min(history.history['val_loss'])

lb = [32, 32, 0.1, 0.00001]
ub = [128, 64, 0.5, 0.01]
best_params_lstm, _ = pso(lstm_objective, lb, ub, swarmsize=10, maxiter=20)

Stopping search: maximum iterations reached --> 20


In [96]:
# Optimized LSTM
lstm_model = Sequential([
    LSTM(int(best_params_lstm[0]), return_sequences=True, input_shape=(12, X_train.shape[2])),
    Dropout(best_params_lstm[2]),
    LSTM(int(best_params_lstm[1])),
    Dropout(best_params_lstm[2]),
    Dense(1)
])
lstm_model.compile(optimizer=RMSprop(learning_rate=best_params_lstm[3]), loss='mse')
early_stop = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
history = lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        epochs=50, batch_size=32, callbacks=[early_stop, reduce_lr], verbose=0)

In [97]:
# Get LSTM predictions
train_pred_lstm = lstm_model.predict(X_train)
val_pred_lstm = lstm_model.predict(X_val)
test_pred_lstm = lstm_model.predict(X_test)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [98]:
# Calculating residuals
train_residuals = y_train - train_pred_lstm.flatten()
val_residuals = y_val - val_pred_lstm.flatten()
test_residuals = y_test - test_pred_lstm.flatten()

In [99]:
# Prepare SVR inputs with lagged residuals
def create_svr_inputs(residuals, original_prices, lags=4, lstm_seq_length=12, forecast_horizon=4):
    X, y = [], []
    X, y = [], []
    for i in range(lags, len(residuals)):
        # Lagged residuals as features for SVR
        lagged_residuals = residuals[i - lags : i]
        price_index_for_residual = i + lstm_seq_length + forecast_horizon - 1
        lagged_prices_for_svr = original_prices[price_index_for_residual - lags : price_index_for_residual]

        # Combining lagged prices and lagged residuals
        X.append(np.concatenate([lagged_prices_for_svr, lagged_residuals]))
        # Targeting for SVR is the current residual
        y.append(residuals[i])

    return np.array(X), np.array(y)
X_svr_train, y_svr_train = create_svr_inputs(train_residuals, train_df['Price'].values, lags=4, lstm_seq_length=12, forecast_horizon=4)
X_svr_val, y_svr_val = create_svr_inputs(val_residuals, val_df['Price'].values, lags=4, lstm_seq_length=12, forecast_horizon=4)
X_svr_test, y_svr_test = create_svr_inputs(test_residuals, test_df['Price'].values, lags=4, lstm_seq_length=12, forecast_horizon=4)

In [100]:
# PSO for SVR optimization
def svr_objective(params):
    C, gamma, epsilon = params
    svr = SVR(kernel='rbf', C=C, gamma=gamma, epsilon=epsilon)
    svr.fit(X_svr_train, y_svr_train)
    val_pred = svr.predict(X_svr_val)
    return mean_squared_error(y_svr_val, val_pred)

lb_svr = [0.1, 0.001, 0.001]
ub_svr = [100, 1.0, 0.5]
best_params_svr, _ = pso(svr_objective, lb_svr, ub_svr, swarmsize=10, maxiter=20)

Stopping search: Swarm best objective change less than 1e-08


In [101]:
# Optimized SVR
svr_model = SVR(kernel='rbf', C=best_params_svr[0], gamma=best_params_svr[1], epsilon=best_params_svr[2])
svr_model.fit(X_svr_train, y_svr_train)

SVR(C=np.float64(100.0), epsilon=np.float64(0.3278017545419454),
    gamma=np.float64(0.6950777410357784))

In [105]:
# Combined predictions
train_pred_svr = svr_model.predict(X_svr_train)
val_pred_svr = svr_model.predict(X_svr_val)
test_pred_svr = svr_model.predict(X_svr_test)
train_pred_combined = train_pred_lstm.flatten()[len(train_pred_lstm.flatten()) - len(train_pred_svr):] + train_pred_svr
val_pred_combined = val_pred_lstm.flatten()[len(val_pred_lstm.flatten()) - len(val_pred_svr):] + val_pred_svr
test_pred_combined = test_pred_lstm.flatten()[len(test_pred_lstm.flatten()) - len(test_pred_svr):] + test_pred_svr

In [106]:
def safe_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = safe_mape(y_true, y_pred)
    return mse, rmse, mae, r2, mape

# Ensuring shapes are aligned
test_pred_lstm = test_pred_lstm.flatten()
lstm_metrics = calculate_metrics(y_test, test_pred_lstm)
y_test_trimmed = y_test[:len(test_pred_combined)]
combined_metrics = calculate_metrics(y_test_trimmed, test_pred_combined)

In [120]:
# Getting baseline LSTM predictions on the test set
baseline_test_pred = baseline_model.predict(X_test)

# Calculating metrics for the baseline LSTM
baseline_lstm_metrics = calculate_metrics(y_test, baseline_test_pred.flatten())

print("Baseline LSTM Metrics:")
print("MSE:", baseline_lstm_metrics[0])
print("RMSE:", baseline_lstm_metrics[1])
print("MAE:", baseline_lstm_metrics[2])
print("R2 Score:", baseline_lstm_metrics[3])
print("MAPE:", baseline_lstm_metrics[4])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Baseline LSTM Metrics:
MSE: 0.027123070457855412
RMSE: 0.1646908329502751
MAE: 0.13114533910471377
R2 Score: -0.25562991903547716
MAPE: 19.551927413578955


In [107]:
#Getting optimized PSO+LSTM and Combinde model metrics
print("LSTM Metrics:")
print("MSE:", lstm_metrics[0])
print("RMSE:", lstm_metrics[1])
print("MAE:", lstm_metrics[2])
print("R2 Score:", lstm_metrics[3])
print("MAPE:", lstm_metrics[4])

print("\nCombined Model Metrics:")
print("MSE:", combined_metrics[0])
print("RMSE:", combined_metrics[1])
print("MAE:", combined_metrics[2])
print("R2 Score:", combined_metrics[3])
print("MAPE:", combined_metrics[4])

LSTM Metrics:
MSE: 0.01896143769492167
RMSE: 0.13770053629133647
MAE: 0.11831919632222296
R2 Score: 0.12220305165430545
MAPE: 15.389657710729388

Combined Model Metrics:
MSE: 0.012022111093198595
RMSE: 0.10964538792488536
MAE: 0.09158360562098918
R2 Score: 0.41661741305572153
MAPE: 11.485935944905883


In [108]:
#Actual vs Predictions
fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=y_test, name='Actual', mode='lines'))
fig1.add_trace(go.Scatter(y=test_pred_lstm.flatten(), name='LSTM', mode='lines'))
fig1.add_trace(go.Scatter(y=test_pred_combined, name='LSTM+SVR', mode='lines'))
fig1.update_layout(title='Actual vs Predicted Prices', xaxis_title='Time', yaxis_title='Price')
fig1.show()

In [110]:
#Residual Scatter Plots
fig2 = go.Figure()
fig2.add_trace(go.Scatter(y=test_residuals, mode='markers', name='LSTM Residuals'))
aligned_test_residuals = test_residuals[len(test_residuals) - len(test_pred_svr):]
fig2.add_trace(go.Scatter(y=aligned_test_residuals - test_pred_svr, mode='markers', name='LSTM+SVR Residuals'))
fig2.update_layout(title='Residuals Scatter Plot', xaxis_title='Index', yaxis_title='Residuals')
fig2.show()

In [113]:
# Residual Histogram
fig3 = go.Figure()
fig3.add_trace(go.Histogram(x=test_residuals, name='LSTM Residuals', opacity=0.5))
aligned_test_residuals = test_residuals[len(test_residuals) - len(test_pred_svr):]
fig3.add_trace(go.Histogram(x=aligned_test_residuals - test_pred_svr, name='LSTM+SVR Residuals', opacity=0.5))
fig3.update_layout(title='Residuals Histogram', xaxis_title='Residuals', yaxis_title='Count', barmode='overlay')
fig3.show()

In [114]:
#Training History
fig4 = go.Figure()
fig4.add_trace(go.Scatter(y=history.history['loss'], name='Train Loss', mode='lines'))
fig4.add_trace(go.Scatter(y=history.history['val_loss'], name='Val Loss', mode='lines'))
fig4.update_layout(title='Training History', xaxis_title='Epoch', yaxis_title='Loss')
fig4.show()

In [123]:
#Metrics Comparison Bar Chart with Baseline
metrics_names = ['MSE', 'RMSE', 'MAE', 'R^2', 'MAPE']
lstm_values = lstm_metrics
combined_values = combined_metrics
baseline_values = baseline_lstm_metrics

fig5 = go.Figure()
fig5.add_trace(go.Bar(x=metrics_names, y=baseline_values, name='Baseline LSTM'))
fig5.add_trace(go.Bar(x=metrics_names, y=lstm_values, name='Optimized LSTM'))
fig5.add_trace(go.Bar(x=metrics_names, y=combined_values, name='LSTM+SVR'))

fig5.update_layout(title='Metrics Comparison', xaxis_title='Metric', yaxis_title='Value', barmode='group')
fig5.show()

In [118]:
print("Summary:")
print(f"LSTM Metrics: MSE={lstm_metrics[0]:.4f}, RMSE={lstm_metrics[1]:.4f}, MAE={lstm_metrics[2]:.4f}, R^2={lstm_metrics[3]:.4f}, MAPE={lstm_metrics[4]:.2f}%")
print(f"LSTM+SVR Metrics: MSE={combined_metrics[0]:.4f}, RMSE={combined_metrics[1]:.4f}, MAE={combined_metrics[2]:.4f}, R^2={combined_metrics[3]:.4f}, MAPE={combined_metrics[4]:.2f}%")

Summary:
LSTM Metrics: MSE=0.0190, RMSE=0.1377, MAE=0.1183, R^2=0.1222, MAPE=15.39%
LSTM+SVR Metrics: MSE=0.0120, RMSE=0.1096, MAE=0.0916, R^2=0.4166, MAPE=11.49%


## Project Summary: Combined LSTM and SVR Model for Time Series Forecasting

This notebook demonstrates building and evaluating a time series forecasting model that combines a Long Short-Term Memory (LSTM) network with Support Vector Regression (SVR) for residual correction.

**Here's a breakdown of the key steps:**

### 1. Data Preparation

*   **Data Loading and Preprocessing:**
    *   Loaded historical rubber futures data.
    *   Converted 'Date' to datetime and sorted data chronologically.
    *   Cleaned and converted 'Vol.' and 'Change %' columns to numeric 'Volume' and 'Change', handling duplicates.
    *   Applied IQR for outlier handling in numerical features.
    *   Included an optional step to subset the data for resource management.

*   **Feature Engineering:**
    *   Created technical indicators (MA, EMA, Bollinger Bands, RSI, Rolling Standard Deviation).
    *   Generated price-based features (High-Low Spread, Price-Open Difference).
    *   Added time-based features (Month, Quarter, Week of Year).

*   **Data Splitting and Scaling:**
    *   Split the data into chronological training, validation, and test sets.
    *   Scaled the features using `MinMaxScaler` based on the training data.

*   **Sequence Creation:**
    *   Transformed the scaled data into 3D sequences suitable for the LSTM input using the `create_sequences` function.

### 2. Model Building and Optimization

*   **LSTM Model:**
    *   Built a baseline LSTM model (without PSO).
    *   Used Particle Swarm Optimization (PSO) to find optimal hyperparameters (units, dropout, learning rate) for the LSTM, minimizing validation loss.
    *   Trained an optimized LSTM model with the best parameters.

*   **SVR Model (for Residual Correction):**
    *   Calculated the residuals (errors) of the optimized LSTM model.
    *   Created SVR input features (`create_svr_inputs`) using lagged residuals and prices. This step involved careful alignment to match the LSTM prediction points.
    *   Used PSO to optimize SVR hyperparameters (C, gamma, epsilon), minimizing the mean squared error on validation residuals.
    *   Trained an optimized SVR model on the training residuals.

### 3. Combining Predictions

*   Combined the predictions by adding the SVR's predicted residuals to the **optimized** LSTM's predictions. This involved aligning the prediction arrays to account for the lagging in SVR input preparation.

### 4. Evaluation and Visualization

*   Evaluated the **baseline LSTM**, the **optimized LSTM**, and the **combined LSTM+SVR** models using standard regression metrics (MSE, RMSE, MAE, R^2, MAPE) on the test set.
*   Visualized the results through:
    *   Actual vs. Predicted Price plots (showing LSTM and Combined).
    *   Residual Scatter Plots (showing LSTM and Combined).
    *   Residual Histograms (showing LSTM and Combined).
    *   Optimized LSTM Training History plot.
    *   Metrics Comparison Bar Chart (comparing all three models).

### 5. Model Comparison Analysis

Based on the evaluation metrics calculated on the test set:

| Metric | Baseline LSTM | Optimized LSTM | Combined LSTM+SVR |
|---|---|---|---|
| MSE | 0.0271 | 0.0190 | 0.0120 |
| RMSE | 0.1647 | 0.1377 | 0.1096 |
| MAE | 0.1311 | 0.1183 | 0.0916 |
| R^2 Score | -0.2556 | 0.1222 | 0.4166 |
| MAPE | 19.55% | 15.39% | 11.49% |

**Analysis:**

*   **Baseline vs. Optimized LSTM:** Comparing the Baseline LSTM to the Optimized LSTM, we can see how the PSO-based hyperparameter optimization impacted the performance of the standalone LSTM model. In this case, the Optimized LSTM shows improvement across most metrics (lower MSE, RMSE, MAE, MAPE, and a positive R^2) compared to the Baseline LSTM.
*   **Optimized LSTM vs. Combined Model:** Comparing the Optimized LSTM to the Combined LSTM+SVR model shows the impact of the residual correction approach using SVR. The Combined Model significantly outperforms the Optimized LSTM across all metrics, achieving a much lower MSE, RMSE, MAE, and MAPE, and a substantially higher R^2 score.

**Conclusion of Comparison:**

Based on the metrics, the **Optimized LSTM performs better than the Baseline LSTM**, supporting the research claim that PSO can improve LSTM performance. Furthermore, the **Combined LSTM and SVR Model demonstrates superior performance** compared to both standalone LSTM models on this dataset, indicating that the residual correction approach is effective in improving forecasting accuracy.

**How the Combined Model Works:**

The core idea is to leverage the strengths of both models. The LSTM excels at capturing sequential dependencies and overall trends in the time series. The SVR is then used to model the patterns in the errors (residuals) left by the **optimized** LSTM. By adding the SVR's predicted residual to the **optimized** LSTM's forecast, the combined model aims to refine the initial prediction and reduce the overall forecasting error, potentially leading to improved accuracy compared to using the standalone optimized LSTM. The PSO optimization helps in finding the best configurations for both models within this combined framework.